# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text = []
    target_id_text = []
    
    for sentence in source_text.split('\n'):
        source_id_text.append([source_vocab_to_int[word] for word in sentence.split()])
    
    for sentence in target_text.split('\n'):
        target_id_text.append([target_vocab_to_int[word] for word in sentence.split()] + [target_vocab_to_int['<EOS>']]) 
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None])
    learn_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, [None], name="target_sequence_length")
    max_target_len = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, [None], name='source_sequence_length')
    
    return input, targets, learn_rate, keep_prob, target_sequence_length, max_target_len, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
              
    end_slice = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    target_data = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), end_slice], 1)

    return target_data

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding

Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    
    embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    def lstm_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(lstm_size, 
                                      initializer=tf.random_uniform_initializer(-0.1, 0.1))
        return tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob)
    
    rnn_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    rnn_cell = tf.contrib.rnn.DropoutWrapper(rnn_cell, output_keep_prob=keep_prob)
    rnn_output, rnn_state = tf.nn.dynamic_rnn(rnn_cell, embed_input, 
                                              sequence_length=source_sequence_length, dtype=tf.float32)
    
    return rnn_output, rnn_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    train_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input, sequence_length=target_sequence_length)
    train_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, train_helper, encoder_state, output_layer)
    
    return tf.contrib.seq2seq.dynamic_decode(train_decoder, impute_finished=True,
                                             maximum_iterations=max_summary_length)[0]



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
    infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                            start_tokens, 
                                                            end_of_sequence_id)
    infer_encoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, infer_helper, encoder_state, output_layer)
    
    return tf.contrib.seq2seq.dynamic_decode(infer_encoder, impute_finished=True,
                             maximum_iterations=max_target_sequence_length)[0]



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
from tensorflow.python.layers.core import Dense

def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    # 2. Decoder Cell
    def lstm_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(lstm_size, initializer=tf.random_uniform_initializer(-0.1, 0.1))
        return tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob)

    dec_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob=keep_prob)

    # 3. Output Layer
    output_layer = Dense(target_vocab_size, kernel_initializer=tf.truncated_normal_initializer(stddev=0.1))

    
    # 4. Training Decoder
    with tf.variable_scope("decode"):
        train_output = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            target_sequence_length, 
                                            max_target_sequence_length, 
                                            output_layer, 
                                            keep_prob)
    
    # 5. Inference Decoder  
    with tf.variable_scope("decode", reuse=True):
        start_of_sequence_id, end_of_sequence_id = target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>']
        infer_output = decoding_layer_infer(encoder_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            start_of_sequence_id, 
                                            end_of_sequence_id, 
                                            max_target_sequence_length, 
                                            target_vocab_size, 
                                            output_layer, 
                                            batch_size, 
                                            keep_prob) 
    
    return train_output, infer_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function   
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  keep_prob, 
                                  source_sequence_length, 
                                  source_vocab_size, 
                                  enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    train_decoder_output, infer_decoder_output = decoding_layer(dec_input, 
                                                                enc_state, 
                                                                target_sequence_length, 
                                                                max_target_sentence_length, 
                                                                rnn_size, 
                                                                num_layers, 
                                                                target_vocab_to_int, 
                                                                target_vocab_size, 
                                                                batch_size, 
                                                                keep_prob, 
                                                                dec_embedding_size)
    
    return train_decoder_output, infer_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [22]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.9
display_step = 10

### Build the Graph
Build the graph using the neural network you implemented.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [25]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   10/1077 - Train Accuracy: 0.2821, Validation Accuracy: 0.3764, Loss: 3.4240
Epoch   0 Batch   20/1077 - Train Accuracy: 0.3457, Validation Accuracy: 0.3896, Loss: 2.9962
Epoch   0 Batch   30/1077 - Train Accuracy: 0.3242, Validation Accuracy: 0.3849, Loss: 2.8212
Epoch   0 Batch   40/1077 - Train Accuracy: 0.3598, Validation Accuracy: 0.4208, Loss: 2.7056
Epoch   0 Batch   50/1077 - Train Accuracy: 0.3730, Validation Accuracy: 0.4538, Loss: 2.5933
Epoch   0 Batch   60/1077 - Train Accuracy: 0.4025, Validation Accuracy: 0.4364, Loss: 2.2735
Epoch   0 Batch   70/1077 - Train Accuracy: 0.3664, Validation Accuracy: 0.4574, Loss: 2.2470
Epoch   0 Batch   80/1077 - Train Accuracy: 0.4242, Validation Accuracy: 0.4968, Loss: 1.9266
Epoch   0 Batch   90/1077 - Train Accuracy: 0.4105, Validation Accuracy: 0.4833, Loss: 1.8593
Epoch   0 Batch  100/1077 - Train Accuracy: 0.4695, Validation Accuracy: 0.5028, Loss: 1.5917
Epoch   0 Batch  110/1077 - Train Accuracy: 0.4629, Validati

Epoch   0 Batch  890/1077 - Train Accuracy: 0.7671, Validation Accuracy: 0.7198, Loss: 0.3137
Epoch   0 Batch  900/1077 - Train Accuracy: 0.7344, Validation Accuracy: 0.7205, Loss: 0.3330
Epoch   0 Batch  910/1077 - Train Accuracy: 0.6920, Validation Accuracy: 0.6776, Loss: 0.3314
Epoch   0 Batch  920/1077 - Train Accuracy: 0.7234, Validation Accuracy: 0.7141, Loss: 0.3344
Epoch   0 Batch  930/1077 - Train Accuracy: 0.7367, Validation Accuracy: 0.7134, Loss: 0.3193
Epoch   0 Batch  940/1077 - Train Accuracy: 0.7316, Validation Accuracy: 0.7163, Loss: 0.3180
Epoch   0 Batch  950/1077 - Train Accuracy: 0.7147, Validation Accuracy: 0.7120, Loss: 0.3031
Epoch   0 Batch  960/1077 - Train Accuracy: 0.7281, Validation Accuracy: 0.6942, Loss: 0.3120
Epoch   0 Batch  970/1077 - Train Accuracy: 0.7191, Validation Accuracy: 0.7493, Loss: 0.3160
Epoch   0 Batch  980/1077 - Train Accuracy: 0.7461, Validation Accuracy: 0.7308, Loss: 0.3189
Epoch   0 Batch  990/1077 - Train Accuracy: 0.6994, Validati

Epoch   1 Batch  700/1077 - Train Accuracy: 0.8855, Validation Accuracy: 0.8540, Loss: 0.1084
Epoch   1 Batch  710/1077 - Train Accuracy: 0.8836, Validation Accuracy: 0.8821, Loss: 0.1117
Epoch   1 Batch  720/1077 - Train Accuracy: 0.8754, Validation Accuracy: 0.8867, Loss: 0.1294
Epoch   1 Batch  730/1077 - Train Accuracy: 0.8809, Validation Accuracy: 0.8746, Loss: 0.1405
Epoch   1 Batch  740/1077 - Train Accuracy: 0.8719, Validation Accuracy: 0.8739, Loss: 0.0977
Epoch   1 Batch  750/1077 - Train Accuracy: 0.8805, Validation Accuracy: 0.8487, Loss: 0.1150
Epoch   1 Batch  760/1077 - Train Accuracy: 0.8867, Validation Accuracy: 0.8750, Loss: 0.1207
Epoch   1 Batch  770/1077 - Train Accuracy: 0.8925, Validation Accuracy: 0.8608, Loss: 0.1130
Epoch   1 Batch  780/1077 - Train Accuracy: 0.8438, Validation Accuracy: 0.8793, Loss: 0.1281
Epoch   1 Batch  790/1077 - Train Accuracy: 0.8586, Validation Accuracy: 0.8718, Loss: 0.1110
Epoch   1 Batch  800/1077 - Train Accuracy: 0.8609, Validati

Epoch   2 Batch  510/1077 - Train Accuracy: 0.8945, Validation Accuracy: 0.9041, Loss: 0.0646
Epoch   2 Batch  520/1077 - Train Accuracy: 0.9580, Validation Accuracy: 0.9041, Loss: 0.0707
Epoch   2 Batch  530/1077 - Train Accuracy: 0.8824, Validation Accuracy: 0.8999, Loss: 0.0744
Epoch   2 Batch  540/1077 - Train Accuracy: 0.9328, Validation Accuracy: 0.9055, Loss: 0.0658
Epoch   2 Batch  550/1077 - Train Accuracy: 0.9000, Validation Accuracy: 0.9102, Loss: 0.0727
Epoch   2 Batch  560/1077 - Train Accuracy: 0.9020, Validation Accuracy: 0.9130, Loss: 0.0687
Epoch   2 Batch  570/1077 - Train Accuracy: 0.9013, Validation Accuracy: 0.8981, Loss: 0.0804
Epoch   2 Batch  580/1077 - Train Accuracy: 0.9453, Validation Accuracy: 0.9034, Loss: 0.0530
Epoch   2 Batch  590/1077 - Train Accuracy: 0.8832, Validation Accuracy: 0.9066, Loss: 0.0818
Epoch   2 Batch  600/1077 - Train Accuracy: 0.9174, Validation Accuracy: 0.9222, Loss: 0.0765
Epoch   2 Batch  610/1077 - Train Accuracy: 0.8886, Validati

Epoch   3 Batch  320/1077 - Train Accuracy: 0.9277, Validation Accuracy: 0.9396, Loss: 0.0617
Epoch   3 Batch  330/1077 - Train Accuracy: 0.9246, Validation Accuracy: 0.9268, Loss: 0.0557
Epoch   3 Batch  340/1077 - Train Accuracy: 0.9301, Validation Accuracy: 0.9208, Loss: 0.0538
Epoch   3 Batch  350/1077 - Train Accuracy: 0.9270, Validation Accuracy: 0.9229, Loss: 0.0486
Epoch   3 Batch  360/1077 - Train Accuracy: 0.9492, Validation Accuracy: 0.9190, Loss: 0.0400
Epoch   3 Batch  370/1077 - Train Accuracy: 0.9446, Validation Accuracy: 0.9197, Loss: 0.0509
Epoch   3 Batch  380/1077 - Train Accuracy: 0.9547, Validation Accuracy: 0.9414, Loss: 0.0411
Epoch   3 Batch  390/1077 - Train Accuracy: 0.8738, Validation Accuracy: 0.9283, Loss: 0.0669
Epoch   3 Batch  400/1077 - Train Accuracy: 0.9316, Validation Accuracy: 0.9293, Loss: 0.0634
Epoch   3 Batch  410/1077 - Train Accuracy: 0.9215, Validation Accuracy: 0.9173, Loss: 0.0581
Epoch   3 Batch  420/1077 - Train Accuracy: 0.9582, Validati

...

Epoch   6 Batch  630/1077 - Train Accuracy: 0.9500, Validation Accuracy: 0.9375, Loss: 0.0348
Epoch   6 Batch  640/1077 - Train Accuracy: 0.9375, Validation Accuracy: 0.9414, Loss: 0.0408
Epoch   6 Batch  650/1077 - Train Accuracy: 0.9730, Validation Accuracy: 0.9457, Loss: 0.0384
Epoch   6 Batch  660/1077 - Train Accuracy: 0.9750, Validation Accuracy: 0.9464, Loss: 0.0365
Epoch   6 Batch  670/1077 - Train Accuracy: 0.9496, Validation Accuracy: 0.9403, Loss: 0.0633
Epoch   6 Batch  680/1077 - Train Accuracy: 0.9375, Validation Accuracy: 0.9393, Loss: 0.0362
Epoch   6 Batch  690/1077 - Train Accuracy: 0.9266, Validation Accuracy: 0.9393, Loss: 0.0403
Epoch   6 Batch  700/1077 - Train Accuracy: 0.9598, Validation Accuracy: 0.9489, Loss: 0.0448
Epoch   6 Batch  710/1077 - Train Accuracy: 0.9469, Validation Accuracy: 0.9510, Loss: 0.0368
Epoch   6 Batch  720/1077 - Train Accuracy: 0.9326, Validation Accuracy: 0.9609, Loss: 0.0614
Epoch   6 Batch  730/1077 - Train Accuracy: 0.9434, Validati

Epoch   7 Batch  440/1077 - Train Accuracy: 0.9449, Validation Accuracy: 0.9311, Loss: 0.0547
Epoch   7 Batch  450/1077 - Train Accuracy: 0.9527, Validation Accuracy: 0.9332, Loss: 0.0626
Epoch   7 Batch  460/1077 - Train Accuracy: 0.9375, Validation Accuracy: 0.9503, Loss: 0.0383
Epoch   7 Batch  470/1077 - Train Accuracy: 0.9704, Validation Accuracy: 0.9421, Loss: 0.0279
Epoch   7 Batch  480/1077 - Train Accuracy: 0.9412, Validation Accuracy: 0.9489, Loss: 0.0329
Epoch   7 Batch  490/1077 - Train Accuracy: 0.9461, Validation Accuracy: 0.9226, Loss: 0.0381
Epoch   7 Batch  500/1077 - Train Accuracy: 0.9605, Validation Accuracy: 0.9474, Loss: 0.0342
Epoch   7 Batch  510/1077 - Train Accuracy: 0.9375, Validation Accuracy: 0.9592, Loss: 0.0479
Epoch   7 Batch  520/1077 - Train Accuracy: 0.9795, Validation Accuracy: 0.9386, Loss: 0.0427
Epoch   7 Batch  530/1077 - Train Accuracy: 0.9277, Validation Accuracy: 0.9336, Loss: 0.0483
Epoch   7 Batch  540/1077 - Train Accuracy: 0.9457, Validati

Epoch   8 Batch  250/1077 - Train Accuracy: 0.9524, Validation Accuracy: 0.9382, Loss: 0.0369
Epoch   8 Batch  260/1077 - Train Accuracy: 0.9461, Validation Accuracy: 0.9286, Loss: 0.0307
Epoch   8 Batch  270/1077 - Train Accuracy: 0.9531, Validation Accuracy: 0.9315, Loss: 0.0489
Epoch   8 Batch  280/1077 - Train Accuracy: 0.9160, Validation Accuracy: 0.9489, Loss: 0.0518
Epoch   8 Batch  290/1077 - Train Accuracy: 0.9398, Validation Accuracy: 0.9418, Loss: 0.0652
Epoch   8 Batch  300/1077 - Train Accuracy: 0.9679, Validation Accuracy: 0.9442, Loss: 0.0278
Epoch   8 Batch  310/1077 - Train Accuracy: 0.9441, Validation Accuracy: 0.9489, Loss: 0.0363
Epoch   8 Batch  320/1077 - Train Accuracy: 0.9437, Validation Accuracy: 0.9577, Loss: 0.0463
Epoch   8 Batch  330/1077 - Train Accuracy: 0.9363, Validation Accuracy: 0.9407, Loss: 0.0458
Epoch   8 Batch  340/1077 - Train Accuracy: 0.9572, Validation Accuracy: 0.9450, Loss: 0.0531
Epoch   8 Batch  350/1077 - Train Accuracy: 0.9613, Validati

Epoch   9 Batch   60/1077 - Train Accuracy: 0.9576, Validation Accuracy: 0.9506, Loss: 0.0310
Epoch   9 Batch   70/1077 - Train Accuracy: 0.9412, Validation Accuracy: 0.9403, Loss: 0.0522
Epoch   9 Batch   80/1077 - Train Accuracy: 0.9570, Validation Accuracy: 0.9393, Loss: 0.0377
Epoch   9 Batch   90/1077 - Train Accuracy: 0.9352, Validation Accuracy: 0.9201, Loss: 0.0443
Epoch   9 Batch  100/1077 - Train Accuracy: 0.9531, Validation Accuracy: 0.9460, Loss: 0.0377
Epoch   9 Batch  110/1077 - Train Accuracy: 0.9586, Validation Accuracy: 0.9407, Loss: 0.0331
Epoch   9 Batch  120/1077 - Train Accuracy: 0.9594, Validation Accuracy: 0.9492, Loss: 0.0571
Epoch   9 Batch  130/1077 - Train Accuracy: 0.9613, Validation Accuracy: 0.9425, Loss: 0.0337
Epoch   9 Batch  140/1077 - Train Accuracy: 0.9490, Validation Accuracy: 0.9407, Loss: 0.0325
Epoch   9 Batch  150/1077 - Train Accuracy: 0.9661, Validation Accuracy: 0.9528, Loss: 0.0409
Epoch   9 Batch  160/1077 - Train Accuracy: 0.9504, Validati

Epoch   9 Batch  940/1077 - Train Accuracy: 0.9613, Validation Accuracy: 0.9620, Loss: 0.0267
Epoch   9 Batch  950/1077 - Train Accuracy: 0.9472, Validation Accuracy: 0.9442, Loss: 0.0365
Epoch   9 Batch  960/1077 - Train Accuracy: 0.9472, Validation Accuracy: 0.9482, Loss: 0.0420
Epoch   9 Batch  970/1077 - Train Accuracy: 0.9547, Validation Accuracy: 0.9371, Loss: 0.0492
Epoch   9 Batch  980/1077 - Train Accuracy: 0.9246, Validation Accuracy: 0.9421, Loss: 0.0386
Epoch   9 Batch  990/1077 - Train Accuracy: 0.9609, Validation Accuracy: 0.9350, Loss: 0.0379
Epoch   9 Batch 1000/1077 - Train Accuracy: 0.9368, Validation Accuracy: 0.9229, Loss: 0.0443
Epoch   9 Batch 1010/1077 - Train Accuracy: 0.9590, Validation Accuracy: 0.9187, Loss: 0.0424
Epoch   9 Batch 1020/1077 - Train Accuracy: 0.9668, Validation Accuracy: 0.9308, Loss: 0.0436
Epoch   9 Batch 1030/1077 - Train Accuracy: 0.9633, Validation Accuracy: 0.9311, Loss: 0.0426
Epoch   9 Batch 1040/1077 - Train Accuracy: 0.9585, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [28]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence = sentence.lower()

    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.split()]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [29]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [182, 163, 202, 19, 148, 77, 95]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [265, 156, 144, 31, 38, 168, 199, 301, 1]
  French Words: il a vu le petit camion blanc . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.